In [11]:
# import dependencies
import pandas as pd
from datasketch import MinHash, MinHashLSH
from tqdm import tnrange, tqdm_notebook
from functools import reduce

In [1]:
# Read files

hdr = pd.read_csv('data1.csv')
items = pd.read_csv('data2.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
print (hdr.head())
print (items.head())

   Unnamed: 0 OLD_INVOICES_KEY INVOICE_ID  ORDER_CUST
0           0    DIST*TT226664   TT226664        6031
1           1    DIST*TT226666   TT226666        5946
2           2    DIST*TT226667   TT226667        5946
3           3    DIST*TT226680   TT226680        6031
4           4    DIST*TT226695   TT226695        5965
   Unnamed: 0   OLD_INVOICES_KEY     LINE_ITEM ORDER_QTY
0           0    PHOENIX*6627207  1639392*1526         1
1           1  PHOENIX*6667405-3    11780*1091       100
2           2    PHOENIX*6558740  1446242*1536         2
3           3    PHOENIX*6558740  1676454*1294         1
4           4    PHOENIX*6514491  1272582*1500        40


In [2]:
# perform inner join

joined = pd.merge(hdr, items, on='OLD_INVOICES_KEY', how='inner')

In [3]:
# drop unnecessary columns

joined.drop(['Unnamed: 0_x','Unnamed: 0_y'],axis=1,inplace=True)

# create product id column
joined['PRODUCT_ID'] = joined['LINE_ITEM'].apply(lambda x: str(x).split('*')[0])

joined.head()

,OLD_INVOICES_KEY,INVOICE_ID,ORDER_CUST,LINE_ITEM,ORDER_QTY,PRODUCT_ID
0,DIST*TN741336,TN741336,6344,4480489*3181,1.0,4480489
1,DIST*TN741336,TN741336,6344,7170928*3259,1.0,7170928
2,DIST*TN741336,TN741336,6344,3320956*3276,1.0,3320956
3,DIST*TN741336,TN741336,6344,3541630*3105,1.0,3541630
4,DIST*TN741347,TN741347,6344,3934347*3349,1.0,3934347


In [58]:
joined['INVOICE_ID'].nunique()

318069

In [77]:
# create dictionary for customers

dic_cust = {}
for val,key in zip(joined['ORDER_CUST'],joined['INVOICE_ID']):
    if key not in dic_cust.keys():
        dic_cust[key] = val

In [69]:
# create dictionary for invoices

dic_invoice = {}
for key,val in zip(joined['INVOICE_ID'],joined['PRODUCT_ID']):
    if key not in dic_invoice.keys():
        dic_invoice[key] = [val]
    else:
        dic_invoice[key].append(val)


In [64]:
def make_df(dic,key,val):
    dict_frame = pd.DataFrame()
    dict_frame[key] = [x for x in dic.keys()]
    dict_frame[val] = [x for x in dic.values()]
    return dict_frame


In [79]:
make_df(dic_cust,'invoices','customer').tail()

,invoices,customer
318064,CL793319,1016659
318065,CL793810,1000822
318066,CL796379,1017903
318067,CL796779,979229
318068,CL796813,1006825


In [70]:
make_df(dic_invoice,'invoice','products').head()

,invoice,products
0,TN741336,"[4480489, 7170928, 3320956, 3541630]"
1,TN741347,[3934347]
2,TN741355,[3050675]
3,TN741370,"[56730, 38130, 11154]"
4,TN741372,"[3635712, 3635386, 3633258, 3634363]"


In [9]:
dict_frame.shape

(318069, 2)

In [12]:
# Create MinHash signatures

num_perm = 128
min_dict = {} # maps invoice id (eg 'TN741347') to min hash signatures

for key,val in tqdm_notebook(zip(dic.keys(),dic.values())): 
    m = MinHash(num_perm=num_perm)
    for shingle in val:
        m.update(shingle.encode('utf8'))
    min_dict[key] = m
    

In [34]:
# Create LSH index

lsh = MinHashLSH(threshold=0.4, num_perm=num_perm)
for key in tqdm_notebook(min_dict.keys()):
    lsh.insert(key,min_dict[key])

In [19]:
# Example bucket of similar invoices

lsh.query(min_dict['TN741336'])

['TN741336']

In [22]:
[x for x in list(min_dict.keys())[:5]]

['TN741336', 'TN741347', 'TN741355', 'TN741370', 'TN741372']

In [91]:
bucket_size = 4
clusters = []
intersection = []
differences = []
prods_leftout = []
customers = []
for val in tqdm_notebook(min_dict.keys()):
    invoice_cluster = lsh.query(min_dict[val])
    leftout_criteria = len(invoice_cluster)/2
    if len(invoice_cluster)>=bucket_size:
        clusters.append(invoice_cluster)
        list_prods = []
        for val2 in invoice_cluster:
            list_prods.append(dic_invoice[val2])
        comm_products = list(reduce(lambda i, j: i & j, (set(x) for x in list_prods)))
        intersection.append(comm_products)
        flat_list = [item for sublist in list_prods for item in sublist]
        customers.append([dic_cust[x] for x in list(set(flat_list))])
        unique_prods = [x for x in flat_list if x not in comm_products]
        differences.append(unique_prods)
        left_out = list(set([x for x in flat_list if flat_list.count(x)>leftout_criteria and flat_list.count(x)<bucket_size]))
        prods_leftout.append(left_out)            
        


In [92]:
output = pd.DataFrame()
output['baskets'] = clusters
output['comm_prods'] = intersection
output['unique_prods'] = differences
output['left_out'] = prods_leftout
output['customers'] = customers
output

,baskets,comm_prods,unique_prods,left_out,customers
0,"[TN999174, TN849574-1, TN879514, TN741355, TN7...",[3050675],"[3046157, 4572731]",[],"[TN978210, TN980439, TN999174]"
1,"[TN854966, TN846046, TN741372, TN816142]",[],"[3635712, 3633258, 3634363, 3632830, 3635243, ...","[3635386, 3635712, 3634363]","[TN996878, TN993153, TN995363, TN981136, TN979..."
2,"[TN832624, TN741392, TN836640, TN800979, TN799...",[1141873],"[1145222, 1683045, 1683045, 1683045, 1145222, ...",[],"[TN980959, TN999001, TN961968, TN981147]"
3,"[TN761155, TN925180, TN986267, TN786646, TN909...",[],"[5176051, 5174419, 5071453, 5176051, 5174338, ...",[],"[TN981446, TN980485-3, TN981446, TN995777, TN9..."
4,"[TN861751, TN995897, TN927272, TN761889, TN936...",[4489121],[4207629],[],"[TN981671, TN981914]"
5,"[TN948740, TN741410, TN763950, TN948800]",[4153186],[4491389],[],"[TN981136, TN959053]"
6,"[TN902187-1, TN902187-3, TN936757, TN957360-5,...",[3892384],"[250429, 3883986, 1628409, 40672, 2246985, 450...",[],"[TN994464, TN978479, TN980538, TN999001, TN957..."
7,"[TN845542, TN978417, TN896049, TN741423, TN761...",[3804954],"[366570, 3805230, 273235]",[],"[TN979245, TN991561, TN972824, TN906601]"
8,"[TN958495, TN823431, TN846320, TN814783, TN741...",[1293180],"[3517810, 2854806]",[],"[TN977941, TN956985, TN976994]"
9,"[TN869337, TN958040, TN893238, TN741477]",[4298720],[],[],[TN993115]
